In [1]:
# temporary solution to crashing
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import mysql.connector as connector
from pathlib import Path

import logging

In [3]:
home = os.path.expanduser('~')
os.chdir(home) # b/c we will be using universal paths

host = '127.0.0.1'
user = 'root' # change to your username
password = 'vasya_is_best_cat_12345' # change to your password
database = 'ai_proj_2025' # for the sake of sanity, we should all have this as the db name 

try:
    conn = connector.connect(
        host = host, 
        user = user, 
        password = password, 
        database = database
    )
    print('success')
except connector.Error as err:
    print(err)

success


Note: the entire CrisisMMD dataset in SQL contains 18,032 images. 

In [4]:
research_dir = Path(home, 'Desktop', 'Education', 'Spring 2025', 'AI', 'research')
os.chdir(research_dir)

from data_helper import SQLDataset_Informative

os.chdir(home)

In [ ]:
#Define the convnet (Eventually replae with mobile 2)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3) #specify the size (outer numbers) and amount (middle number) of filters
        self.pool = nn.MaxPool2d(2, 2) #specify pool size first number is size of pool, second is step size
        self.conv2 = nn.Conv2d(16, 8, 3) #new depth is amount of filters in previous conv layer
        self.fc1 = nn.Linear(54*54*8, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 3) #finial fc layer needs 19 outputs because we have 19 layers # ???

    def forward(self, x):

        x = F.relu(self.conv1(x))
     
        x = self.pool(x)
       
        x = F.relu(self.conv2(x))

        x = self.pool(x)

        x = x.view(-1, 54*54*8) # flatten

        x = F.relu(self.fc1(x))    #fully connected, relu         
        x = F.relu(self.fc2(x))    
       
        x = self.fc3(x)     #output    
        return x

In [6]:
from torchvision.transforms import v2

# transforms
transformations = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224), antialias=True), 
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
set = SQLDataset_Informative(conn=conn, img_col='image_path', label_col='image_info', transform=transformations, 
                     data_dir=Path('OneDrive - Stephen F. Austin State University', 'CrisisMMD_v2.0','CrisisMMD_v2.0'))

In [8]:
from torch.utils.data import DataLoader
train_loader = DataLoader(set, batch_size=32)

In [9]:
model = ConvNet()

In [10]:
test_result = model.forward(set.__getitem__(0)[0])
test_result.shape

torch.Size([1, 3])

In [11]:
learning_rate = .001
num_epochs = 1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from tqdm import tqdm 

#Train the model
model = ConvNet().to(device)

cross_entropy = nn.CrossEntropyLoss() #cross entropy loss for our loss function
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #stocastic gradient descent for our optimization algorithm

for epoch in range(num_epochs):
    print('Epoch', epoch)
    for item in tqdm(train_loader, total= len(train_loader), desc="Processing data in epoch {epoch}"):

        images = item[0]

        labels = item[1]
        assert(labels.shape[0] == images.shape[0])

        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model.forward(images)

        loss = cross_entropy(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward() # backpropagation
        optimizer.step() # updating parameters
        
print('Finished Training')
PATH = research_dir / './cnn.pth'
torch.save(model.state_dict(), PATH)

Epoch 0


Processing data in epoch {epoch}: 100%|██████████| 566/566 [1:25:57<00:00,  9.11s/it]

Finished Training


In [18]:
loaded_model_dict = torch.load(PATH)

C:\Users\agafi\AppData\Local\Temp\ipykernel_16504\4220502338.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model_dict = torch.load(PATH)


In [20]:
loaded_model_dict.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])

In [23]:
loaded_model = ConvNet().load_state_dict(state_dict=loaded_model_dict)